In [1]:
parent_folder = '/Users/ravi.tej/Desktop/ML/Recommendations/hubble/'
from hydra import compose, initialize
import os

import xml.etree.ElementTree as ET

tree = ET.parse('../../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

import sys
sys.path.append('/Users/ravi.tej/Desktop/ML/Recommendations/hubble/')

In [2]:
from src.RAGTraining.TrainingDataGenerator import TrainingDataGenerator

/Users/ravi.tej/anaconda3/envs/recoenv/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

In [4]:
trainingGenerator = TrainingDataGenerator(model_id = 'teknium/OpenHermes-2.5-Mistral-7B', queries_df_path='insight_user_db_publarticles_search_audit.csv', responses_file_path='gpt-responses.csv')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
trainingGenerator.content_tokens

3923

In [6]:
import json

with open('training_article_ids_latest.json', 'r') as f:
    best_results = json.load(f)
trainingGenerator.best_results = best_results

with open('training_user_prompts_latest.json', 'r') as f:
    user_prompts = json.load(f)
trainingGenerator.user_prompts = user_prompts

In [7]:
len(trainingGenerator.user_prompts)

1122

In [8]:
# trainingGenerator.generate_prompts_for_queries()
# trainingGenerator.save_results_to_json(trainingGenerator.user_prompts, 'training_user_prompts_latest.json')

In [9]:
# # trainingGenerator.get_best_articles_for_queries(max_num_random_dates=3, randomize_num_dates_per_query=True)

# len(trainingGenerator.best_results)

# # trainingGenerator.save_results_to_json(trainingGenerator.best_results, 'training_article_ids_latest.json')

# trainingGenerator.generate_prompts_for_queries()

# trainingGenerator.save_results_to_json(trainingGenerator.user_prompts, 'training_user_prompts_latest.json')

In [27]:
trainingGenerator.get_gpt4_attributes_for_article_ids(fraction=1)


KeyboardInterrupt



### Cleaning

In [89]:
import pandas as pd

In [90]:
responses_df = pd.read_csv('gpt-responses.csv',  header=None)
responses_df = responses_df.iloc[:1121]

In [3]:
def has_hash(x):
    return '##' in x

In [4]:
responses_df['hash'] = responses_df[6].apply(lambda x: has_hash(x))

In [5]:
responses_df.hash.mean()

0.012488849241748439

In [6]:
import re

def replace_br_tags(text):
    # This pattern matches sequences of more than two <br> tags.
    # The pattern looks for <br> followed by one or more repetitions of <br>, capturing as many as it finds.
    pattern = r'(<br>\s*){3,}'
    # Replacement string with exactly two <br> tags.
    replacement = r'<br><br>'
    # Use re.sub to replace sequences of more than two <br> tags with exactly two.
    modified_text = re.sub(pattern, replacement, text)
    return modified_text

In [7]:
def clean_response(x):
    x = x.replace('\\n','<br>')
    x = x.replace('\n', '<br>')
    x = x.replace('####', '**')
    x = x.replace('###', '**')
    x = x.replace('##', '**')
    x = x.replace('#', '**')
    x = replace_br_tags(x)
    return x

In [8]:
import json

In [9]:
responses_df['cleaned_response'] = responses_df[6].apply(lambda x: json.loads(clean_response(x)))

In [10]:
import re

def extract_headline(text):
    # Regular expression to match the headline pattern
    pattern = r'\*\*(.*?)\*\*(?::)?(?:\s*<br>\s*){1,2}'
    match = re.search(pattern, text, re.IGNORECASE | re.DOTALL)
    if match:
        headline = match.group(1).strip()
        rest_of_text = text[match.end():].strip()
        return headline, rest_of_text
    return None, text

# Example usage
text = "**This is the headline** <br><br> Here is the rest of the text."
headline, rest_of_text = extract_headline(text)
print("Headline:", headline)
print("Rest of the text:", rest_of_text)


Headline: This is the headline
Rest of the text: Here is the rest of the text.


In [12]:
#if x['has_headline'] else ('', x['cleaned_response'])
responses_df['headline'], responses_df['headline_removed_responses'] = zip(*responses_df.apply(lambda x: extract_headline(x['cleaned_response']) ,axis=1))

In [13]:
responses_df.headline.isnull().mean()

0.32917038358608386

In [14]:
def remove_hyphen_from_section_headings(text):
    # Regular expression to match the pattern
    pattern = r'(<br>\s*)?-\s*\*\*([^<\n]*?)\*\*(\s*:\s*)?(?=(\s*<br>|$))'
    def replace_func(match):
        return f"{match.group(1) or ''}**{match.group(2)}{match.group(3) or ''}**{match.group(4) or ''}"
    updated_text = re.sub(pattern, replace_func, text, flags=re.DOTALL)
    return updated_text

In [15]:
def replace_developed_by_openai(x):
    if 'AI developed by OpenAI' in x:
        return x.replace('AI developed by OpenAI', 'AI developed by Insight')
    else:
        return x

In [16]:
def clean_provided_articles(x):
    if 'provided articles' in x: 
        return x.replace('provided articles', 'articles I found')
    elif 'articles provided' in x:
        return x.replace('articles provided', 'articles I found')
    elif 'information provided' in x:
        return x.replace('information provided', 'articles I found')
    else:
        return x

In [24]:
def remove_citations(x):
    x = x.replace('{{1}}', '')
    x = x.replace('{{2}}', '')
    x = x.replace('{{3}}', '')
    return x

In [31]:
def clean_responses_further(x):
    x = remove_hyphen_from_section_headings(x)
    x = replace_developed_by_openai(x)
    x = clean_provided_articles(x)
    # x = remove_citations(x)
    return x

In [32]:
def has_references(x):
    return '{{1}}' in x or '{{2}}' in x or '{{3}}' in x

In [33]:
def find_apologies_responses(x):
    return 'I\'m sorry' in x or 'apologize' in x or 'here to provide' in x or 'here to assist' in x

In [34]:
responses_df.headline_removed_responses.sample(30).values

array(["The Indian economy has shown resilience in the face of global economic headwinds, including conflicts and restrictive US monetary policies. Here are the key highlights and future outlook based on recent reviews:<br><br>- **Current Economic Climate**:<br>  - Conflicts in the Persian Gulf and the US's restrictive monetary stance pose risks, potentially affecting oil prices and borrowing costs {{1}}.<br>  - Despite these challenges, India's FY24 economic outlook remains positive, supported by strong domestic fundamentals and a decrease in retail inflation {{1}}.<br><br>- **Growth Projections**:<br>  - The Indian economy is expected to grow at a rate close to 7% in FY25, underpinned by robust domestic demand driven by private consumption and investment {{2}}.<br>  - There's potential for the growth rate to exceed 7% by 2030, backed by infrastructure development, balance sheet strengthening, and technological advancements {{2}}.<br><br>- **Strategic Reforms and Economic Aspirations*

In [35]:
responses_df['cleaned_headline_removed_responses'] = responses_df['headline_removed_responses'].apply(lambda x: clean_responses_further(x))

In [36]:
responses_df['apologies'] = responses_df['cleaned_headline_removed_responses'].apply(lambda x: find_apologies_responses(x))

In [37]:
responses_df['has_ref'] = responses_df['cleaned_headline_removed_responses'].apply(lambda x: has_references(x))

In [38]:
responses_df.has_ref.mean()

0.6975914362176628

### Identifying errenous cases

In [39]:
responses_df.apologies.mean()

0.28278322925958965

In [40]:
len(responses_df[responses_df.headline.isnull()])

369

In [45]:
responses_df[responses_df.headline.isnull()].sample(10)['cleaned_headline_removed_responses'].values

array(['I\'m sorry, but it appears there was a misunderstanding in your query. You mentioned "how to pan my taxes," which seems unclear. If you were asking about how to obtain or manage your Permanent Account Number (PAN) details for tax-related purposes in India, the available articles provide extensive guidance on how to access, retrieve, or correct your PAN card details both online and offline.<br><br>However, if your query is specifically about planning or managing your taxes using your PAN, such direct advice on tax planning or management was not covered directly in the articles I found. <br><br>Could you clarify or provide more detail about your query? Are you looking for information on accessing PAN details, correcting PAN details, or specific guidance on tax planning and management with respect to your PAN?',
       "I'm sorry, but it seems I can't assist with providing jokes. My primary focus is on offering information and answers related to finance and business topics. If you

In [46]:
len(responses_df[(responses_df.has_ref == False) & (responses_df.apologies == False)])

41

In [59]:
def has_question(x):
    return '?' in x and '**' not in x

In [60]:
responses_df['has_question'] = responses_df['cleaned_headline_removed_responses'].apply(lambda x: has_question(x))

In [61]:
responses_df.has_question.mean()

0.058876003568242644

In [62]:
responses_df[responses_df.has_question == True]['cleaned_headline_removed_responses'].sample(20).values

array(["I'm sorry, but it seems your query is missing or unclear. Could you please provide more details or specify your question? I'm here to help with finance and business-related inquiries.",
       'I\'m sorry, but it seems there might have been a misunderstanding or a typographical error in your query. When you mentioned "Is fd a destroyer?" it appears you might be referring to financial deposits (FDs) or something else entirely, but your query was matched with articles related to military naval destroyers. Could you please provide more details or clarify your question? If you\'re asking about financial products or India\'s financial environment, I\'d be more than happy to assist with the correct information.',
       'I\'m sorry, but it seems like there was a misunderstanding in the query input. You mentioned "smart koney," which might have been a typographical error. Could you clarify or correct your query if you meant to inquire about "smart money," or if it relates to finance a

In [47]:
responses_df[(responses_df.has_ref == False) & (responses_df.apologies == False)].cleaned_headline_removed_responses.values

array(["I'm unable to use the given articles to generate responses as they do not contain any user-provided information to reference directly.",
       "I'm unable to find a direct statement or analysis by Sanjiv Bhasin regarding RBI’s directive on Paytm being a big blow or his recommendations on three dark horse picks for investors. My resources primarily discuss the financial penalties imposed on Paytm by the RBI for KYC non-compliance, the subsequent market reaction, Paytm's share performance, and strategic shifts within the company. For investment advice or specific analysts' picks not covered in these articles, I recommend consulting the latest financial news or directly accessing expert analyses from reputable financial advisory services.",
       "The regular trading hours for the Indian stock market, which includes the Bombay Stock Exchange (BSE) and the National Stock Exchange (NSE), are from 9:15 AM to 3:30 PM Indian Standard Time (IST) from Monday to Friday. Additionally, th

In [63]:
def flag_responses(row):
    if (row['has_ref'] == False) and (row['apologies'] == False):
        return True
    elif row['has_question']:
        return True
    return False

In [64]:
responses_df['flagged'] = responses_df.apply(lambda x: flag_responses(x),axis=1)

In [65]:
responses_df.groupby('flagged')['flagged'].count()

flagged
False    1020
True      101
Name: flagged, dtype: int64

In [66]:
responses_df[responses_df['flagged'] == False]['cleaned_headline_removed_responses'].sample(30).values

array(["I'm sorry, but as an AI developed by Insight, I don't provide personal opinions or evaluations of political figures, including Prime Minister Narendra Modi. However, I can share that Union Minister of State for Chemicals & Fertilizers Bhagwanth Khuba praised the economic growth under Modi's leadership, attributing it to a stable government and policy changes aimed at self-reliance and attracting investors {{1}}. Additionally, Prime Minister Modi himself emphasized India's resilience and growth potential amidst global economic challenges {{2}}. He also highlighted India's digital transformation and its role in fostering a global digital economy during his address at the G20 Digital Economy Ministerial Meet {{3}}. These perspectives provide insight into the official stance on economic progress and digital innovation during Modi's tenure.",
       "- Wipro has expanded its partnership with Google Cloud to advance the adoption of generative AI across global enterprises. This collab

In [70]:
responses_df[responses_df['flagged'] == False].headline.isnull().mean()

0.2803921568627451

### Removing citations

In [71]:
responses_df[3] = '''You are an Indian finance and business expert tasked with providing accurate and concise responses to finance and business-related queries, particularly those involving key entities like India, RBI, Reliance, and the Fed. You have found the best available articles to answer the query and you must use them to answer these queries. Use the best articles available to answer queries, ensuring your responses are tailored to the query's requirements. Ensure that your answers are tailored to the query's requirements, employing only one of the following structured formats based on the query

1. **Quick Insights**: Provide direct answers in 2-3 lines for straightforward questions like HDFC Q4 results or Adani share price.
2. **Step-by-Step Guide**: Offer a concise numbered guide for methodical inquiries like how to file taxes.
3. **Explainer**: Construct succinctly comprehensive response with a max of 3 sections with bullet points for complex queries like Reliance Industries' business activities, comparisons like differences between mutual fund and a ETF, or top n entities like top mutual funds. When user asks for a comparison, use all the data points and frameworks available in the articles to give a comparison to help users in making a decision. 

Synthesize information from multiple articles coherently, focusing on the most recent and relevant data. If a query cannot be fully addressed but related information is available, provide a succinct answer. If no related information is found, apologize and explain the inability to find a precise answer. Decline non-financial queries politely, emphasizing the portal's finance and business focus.

**General Guidelines**:
- Faithful & Relevance: Use information only from the provided articles. Respond with the most important information first. The response should directly answer the user query.
- Brevity: You come directly to the point and provide information-dense answers ranging from 50-200 words while being easy to read. Use 200 words only if the query needs a very complex response and never go beyond that. Do not explain unnecessary basics. Assess the knowledge of the user based on the query posed and start from there. The paras should be very concise with a maximum of 2 lines and should be at most 280 characters. Do not use any headline or introduction. Directly answer.
- Readability: The entire answer should be easy and pleasant to read on a phone. Use a mix of paras and points with appropriate line breaks and sections. If using sections, limit them to 3. Use ** for bolding section headers and <br> for line breaks. Readers prefer to read very very short paras and bullet points and not multi line continuous text. 
- Focus: Focus on Indian information unless otherwise specified.
- Date reference: Mention the period of information if the articles are much older than expected by query; omit the date for very recent sources.

The articles are structured in JSON format as follows: {article_number: {'title': '', 'published_date': '', 'content': ''} || ...}'''

In [72]:
def remove_citations(x):
    x = x.replace('{{1}}', '')
    x = x.replace('{{2}}', '')
    x = x.replace('{{3}}', '')
    return x

In [73]:
responses_df['cleaned_responses_without_citations'] = responses_df['cleaned_headline_removed_responses'].apply(lambda x: remove_citations(x))

In [74]:
responses_df[responses_df.apologies == True][8].mean()

75.26813880126183

In [76]:
def remove_spaces_before_period(text):
    # Use regular expression to remove any spaces before a period
    updated_text = re.sub(r'\s+\.', '.', text)
    return updated_text

In [77]:
responses_df['cleaned_responses_without_citations'] = responses_df['cleaned_responses_without_citations'].apply(lambda x: remove_spaces_before_period(x))

In [81]:
responses_df[responses_df.flagged == False]['cleaned_responses_without_citations'].sample(30).values

array(["A dead cat bounce is a temporary phenomenon in the stock market, often misinterpreted by traders as a reversal of a downward trend. Here's a breakdown of its key aspects:<br><br>**Definition and Characteristics:**<br><br>  - A dead cat bounce occurs when there's a short-lived recovery in the price of a declining asset.<br>  - Despite the temporary rise, the downtrend soon resumes, making this pattern a continuation pattern rather than a reversal.<br>  - Identifying a dead cat bounce can be challenging in real-time and is often only confirmed post-event.<br><br>**Implications for Traders:**<br><br>  - It's risky for new traders, as it may deceive them into believing a bullish trend is underway.<br>  - Understanding and identifying dead cat bounces can help traders make informed decisions, such as maintaining short positions instead of cutting them short prematurely.<br><br>**How to Spot and Trade:**<br><br>  - Early signs include a brief rally on low volume and a lower high comp

### Save cleaned responses

In [82]:
len(responses_df)

1121

In [83]:
len(responses_df[responses_df.flagged == False])

1020

In [84]:
# responses_df[responses_df.flagged == False].to_csv('cleaned_responses.csv',index=False)

In [85]:
responses_df = responses_df.rename(columns = {1: 'query', 2: 'query_date', 3: 'system_prompt', 4: 'user_prompt', 5: 'model', 6: 'model_response', 7: 'input_tokens', 8: 'output_tokens'})

In [86]:
responses_df

,0,query,query_date,system_prompt,user_prompt,model,model_response,input_tokens,output_tokens,hash,cleaned_response,headline,headline_removed_responses,cleaned_headline_removed_responses,apologies,has_ref,flagged,has_question,cleaned_responses_without_citations
0,i,Unitec,2024-01-26T18:06:29,You are an Indian finance and business expert ...,|| user_query: Unitec || today date: 26 Janua...,gpt-4-0125-preview,"""I'm sorry, but there doesn't seem to be any i...",1593,67,False,"I'm sorry, but there doesn't seem to be any in...",None,"I'm sorry, but there doesn't seem to be any in...","I'm sorry, but there doesn't seem to be any in...",True,False,False,False,"I'm sorry, but there doesn't seem to be any in..."
1,i,Sjadkjhfjkdshfkjsdhjkfhdsjkfdshfkdshjkfdshkjfsd,2024-01-19T23:57:34,You are an Indian finance and business expert ...,|| user_query: Sjadkjhfjkdshfkjsdhjkfhdsjkfds...,gpt-4-0125-preview,"""I'm sorry, but it seems like your query didn'...",1403,48,False,"I'm sorry, but it seems like your query didn't...",None,"I'm sorry, but it seems like your query didn't...","I'm sorry, but it seems like your query didn't...",True,False,True,True,"I'm sorry, but it seems like your query didn't..."
2,i,Sensex,2024-01-17T00:15:05,You are an Indian finance and business expert ...,|| user_query: Sensex || today date: 17 Janua...,gpt-4-0125-preview,"""**Quick Insights**\n\nAs of the latest update...",3363,92,False,**Quick Insights**<br><br>As of the latest upd...,Quick Insights,"As of the latest update on December 18, 2023, ...","As of the latest update on December 18, 2023, ...",False,True,False,False,"As of the latest update on December 18, 2023, ..."
3,i,Best shawarma in India,2024-01-03T22:10:44,You are an Indian finance and business expert ...,|| user_query: Best shawarma in India || toda...,gpt-4-0125-preview,"""I'm sorry, but I'm unable to fulfill this req...",5423,13,False,"I'm sorry, but I'm unable to fulfill this requ...",None,"I'm sorry, but I'm unable to fulfill this requ...","I'm sorry, but I'm unable to fulfill this requ...",True,False,False,False,"I'm sorry, but I'm unable to fulfill this requ..."
4,i,Best shawarma in India,2024-01-03T22:10:44,You are an Indian finance and business expert ...,|| user_query: Best shawarma in India || toda...,gpt-4-0125-preview,"""I'm sorry, but I don't have any information o...",5423,50,False,"I'm sorry, but I don't have any information on...",None,"I'm sorry, but I don't have any information on...","I'm sorry, but I don't have any information on...",True,False,False,False,"I'm sorry, but I don't have any information on..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1116,i,what is 10 10 d section,2024-01-07T10:44:19,You are an Indian finance and business expert ...,|| user_query: what is 10 10 d section || to...,gpt-4-0125-preview,"""**Quick Insights**\n\nSection 10 (10D) of the...",6088,126,False,**Quick Insights**<br><br>Section 10 (10D) of ...,Quick Insights,Section 10 (10D) of the Income Tax Act exempts...,Section 10 (10D) of the Income Tax Act exempts...,False,True,False,False,Section 10 (10D) of the Income Tax Act exempts...
1117,i,Help me select a tax regime in 2024,2024-01-04T04:08:36,You are an Indian finance and business expert ...,|| user_query: Help me select a tax regime in...,gpt-4-0125-preview,"""Choosing between the old and new income tax r...",2381,309,False,Choosing between the old and new income tax re...,Quick Insights,- **New Tax Regime (From FY 2023-24 onwards)**...,- **New Tax Regime (From FY 2023-24 onwards)**...,False,True,False,False,- **New Tax Regime (From FY 2023-24 onwards)**...
1118,i,Wint blogs,2024-01-10T22:59:23,You are an Indian finance and business expert ...,|| user_query: Wint blogs || today date: 10 J...,gpt-4-0125-preview,"""**Quick Insights**\n\nWint Wealth, an online ...",2468,153,False,"**Quick Insights**<br><br>Wint Wealth, an onli...",Quick Insights,"Wint Wealth, an online bond platform, is expan...","Wint Wealth, a

In [87]:
responses_df = responses_df.drop(columns = [0, 'hash'])

In [88]:
responses_df[responses_df.flagged == False].to_csv('gpt-4-cleaned_responses.csv',index=False)

In [54]:
import re

def add_space_after_unicode_representation(text):
    # Pattern to find any Unicode representation (e.g., \u20b9 or \U000020b9) followed directly by digits.
    # This pattern matches both short (\uxxxx) and long (\Uxxxxxxxx) Unicode representations.
    pattern = r'(\\u[0-9a-fA-F]{4}|\\U[0-9a-fA-F]{8})(\d)'

    # Replacement pattern with a space between the Unicode representation and the number.
    replacement = r'\1 \2'

    # Use re.sub to replace the pattern with our desired format.
    modified_text = re.sub(pattern, replacement, text)

    return modified_text

In [47]:
import json

In [75]:
(responses_df['input_tokens'].sum() * 0.01)/1000 + (responses_df['output_tokens'].sum() * 0.03)/1000

47.66874

In [76]:
responses_df['output_tokens'].describe()

count    1121.000000
mean      198.030330
std       131.018691
min         9.000000
25%        92.000000
50%       157.000000
75%       299.000000
max       641.000000
Name: output_tokens, dtype: float64